In [1]:
import tensorflow as tf
import pandas as pd
import numpy as np

In [2]:
df = pd.read_json('data/Movies_and_TV_5.json', lines=True)
X = df['reviewText'].values
y = df['overall'].values
print(f"X.shape: {X.shape}")
print(f"y.shape: {y.shape}")
y = y - 1
# set(y)

In [24]:
X = df['reviewText'].to_numpy(dtype='<U2000')

In [25]:
VOCAB_SIZE = 30000
tf.keras.backend.clear_session()
binary_vectorizer = tf.keras.layers.TextVectorization(
    max_tokens=VOCAB_SIZE,
    output_mode='count',
    pad_to_max_tokens=True
)
binary_vectorizer.adapt(X)


In [182]:
model = tf.keras.Sequential([
    tf.keras.Input(shape=(1,), dtype=tf.string),
    binary_vectorizer,
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(16, activation='relu'),
    tf.keras.layers.Dense(5, activation='softmax')
])
model.compile(optimizer=tf.keras.optimizers.Adam(),
                loss=tf.keras.losses.SparseCategoricalCrossentropy(),
                metrics=['accuracy'])

In [183]:
model.fit(X,y, epochs=50)

Epoch 1/50
401/401 [==============================] - 2s 4ms/step - loss: 1.1206 - accuracy: 0.5590
Epoch 2/50
401/401 [==============================] - 2s 4ms/step - loss: 0.7939 - accuracy: 0.6844
Epoch 3/50
401/401 [==============================] - 2s 4ms/step - loss: 0.5717 - accuracy: 0.7804
Epoch 4/50
401/401 [==============================] - 2s 4ms/step - loss: 0.3897 - accuracy: 0.8607
Epoch 5/50
401/401 [==============================] - 2s 4ms/step - loss: 0.2590 - accuracy: 0.9137
Epoch 6/50
401/401 [==============================] - 2s 4ms/step - loss: 0.1866 - accuracy: 0.9425
Epoch 7/50
401/401 [==============================] - 2s 4ms/step - loss: 0.1284 - accuracy: 0.9614
Epoch 8/50
401/401 [==============================] - 2s 4ms/step - loss: 0.1012 - accuracy: 0.9716
Epoch 9/50
401/401 [==============================] - 2s 4ms/step - loss: 0.0987 - accuracy: 0.9699
Epoch 10/50
401/401 [==============================] - 2s 4ms/step - loss: 0.1181 - accuracy: 0.9662

KeyboardInterrupt: 

In [184]:
model.summary()

Model: "sequential_22"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
text_vectorization (TextVect (None, 5000)              0         
_________________________________________________________________
dense_35 (Dense)             (None, 64)                320064    
_________________________________________________________________
dense_36 (Dense)             (None, 16)                1040      
_________________________________________________________________
dense_37 (Dense)             (None, 5)                 85        
Total params: 321,189
Trainable params: 321,189
Non-trainable params: 0
_________________________________________________________________


In [21]:
import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  1
